In [7]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd

file_path = 'files/csvs/Biscuits and Sausage Gravy_sheet0.csv'

with open(file_path, 'r') as f:
    all_lines = f.readlines()

header_row_index = -1
for i, line in enumerate(all_lines):
    if line.strip().startswith('ingredients:'):
        header_row_index = i
        break

if header_row_index == -1:
    raise ValueError("Could not find the 'ingredients:' header row in the file.")

recipe_row_starting = -1
for i, line in enumerate(all_lines):
    if line.strip().startswith('cooking instructions:'):
        recipe_row_starting = i
        break

if recipe_row_starting == -1:
    raise ValueError("Could not find the 'cooking instructions:' row in the file.")


metadata_lines = all_lines[:header_row_index]
metadata = {}
for line in metadata_lines:
    parts = line.split(',')
    try:
        if 'Unnamed: 5' in line:
            metadata['target_name'] = parts[6].strip()
        elif 'source:' in line:
            metadata['source'] = parts[16].strip()
        elif 'region:' in line:
            metadata['region'] = int(parts[16].strip())
    except (IndexError, ValueError):
        continue

ingredients_df = pd.read_csv(
    file_path,
    header=None,
    skiprows=header_row_index + 1
)
instruction_idx = recipe_row_starting + 1
while instruction_idx < len(all_lines):
    candidate_line = all_lines[instruction_idx]
    if candidate_line.strip() and not all(part.strip() == '' for part in candidate_line.strip().split(',')):
        instruction_line = candidate_line
        next_idx = instruction_idx + 1
        if next_idx < len(all_lines):
            next_line = all_lines[next_idx]
            if next_line.strip() and not all(part.strip() == '' for part in next_line.strip().split(',')):
                instruction_line = instruction_line.rstrip().rstrip(',') + ' ' + next_line.lstrip().lstrip(',')
        break
    instruction_idx += 1
else:
    raise ValueError("can't find non-empty line after 'cooking instructions:'.")


if not ingredients_df.empty and max([2, 9, 11]) < len(ingredients_df.columns):
    df_useful = ingredients_df[[2, 9, 11]]
    df_useful.columns = ['ingredient', 'quantity', 'unit']
    ingredients_df = df_useful.dropna(subset=['ingredient']).reset_index(drop=True)
    numeric_cols = ['quantity']
    ingredients_df[numeric_cols] = ingredients_df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    ingredients_df['unit'] = ingredients_df['unit'].fillna('item')
else:
    ingredients_df = pd.DataFrame(columns=['ingredient', 'quantity', 'unit'])

if not ingredients_df.empty:
    ingredients_df = ingredients_df.groupby('ingredient', as_index=False).agg({
        'quantity': 'sum',
        'unit': 'first'
    })

try:
    recipe_text = instruction_line.split(',')[0].strip()
except IndexError:
    recipe_text = "Recipe text not foun in the expected column."

df_recipe = pd.DataFrame({'recipe': [recipe_text]})

print("Extracted Data")
for key, value in metadata.items():
    if key == "target_name":
        print(f"Dish Name: {value}")

print("\n" + "Extracted Ingerdients")
print(ingredients_df)
print(' '.join(instruction_line.strip().rstrip(',').split()))

Extracted Data
Dish Name: Biscuits and Sausage Gravy

Extracted Ingerdients
               ingredient  quantity   unit
0                Biscuits      8.00     ct
1  Bulk Breakfast Sausage      1.00    lbs
2            Eggs - fried      8.00     ct
3                    Milk      3.00    cup
4                  Pepper      1.00  taste
5                    Roux      0.25    cup
6                    Salt      1.00  taste
"Cut the biscuits in half. Brown the sausage. Thicken the milk with roux. Add it to the sausage. Add pepper, adjust salt and serve the gravy over the biscuits. Top with a fried egg."


In [ ]:
import psycopg2
import pandas as pd

db_params = {
    'dbname': 'ristorante',
    'user': 'headchef',
    'password': '123',
    'host': '127.0.0.1',
    'port': '5432'
}

def get_or_create_id(cur, table, id_col, name_col, name_value):
    cur.execute(f"SELECT {id_col} FROM {table} WHERE {name_col} = %s", (name_value,))
    result = cur.fetchone()

    if result:
        return result[0]
    else:
        print(f"'{name_value}' not found in '{table}'. Creating new entry...")
        cur.execute(f"INSERT INTO {table} ({name_col}) VALUES (%s) RETURNING {id_col}", (name_value,))
        new_id = cur.fetchone()[0]
        return new_id


recipe_name = metadata.get('target_name', 'Unnamed Recipe')
instructions = ' '.join(instruction_line.strip().rstrip(',').split())

conn = None
try:
    print("Connecting to the database...")
    conn = psycopg2.connect(**db_params)

    with conn.cursor() as cur:
        sql_check_exists = "SELECT EXISTS(SELECT 1 FROM recipes WHERE recipe_name = %s)"
        cur.execute(sql_check_exists, (recipe_name,))
        recipe_exists = cur.fetchone()[0]

        if recipe_exists:
            print(f"'{recipe_name}' exists already.")
        
        else:
            print(f"'{recipe_name}' doesn't exist yet.")
            
            sql_insert_recipe = """
                INSERT INTO recipes (recipe_name, instructions)
                VALUES (%s, %s)
                RETURNING recipe_id;
            """
            cur.execute(sql_insert_recipe, (recipe_name, instructions))
            new_recipe_id = cur.fetchone()[0]
            print(f"Created new recipe with ID: {new_recipe_id}")

            if not ingredients_df.empty:
                print("Inserting ingredients...")
                for index, row in ingredients_df.iterrows():
                    ingredient_id = get_or_create_id(cur, 'ingredients', 'ingredient_id', 'ingredient_name', row['ingredient'])
                    unit_id = get_or_create_id(cur, 'units', 'unit_id', 'unit_name', row['unit'])
                    
                    sql_insert_link = """
                        INSERT INTO recipe_ingredients (recipe_id, ingredient_id, quantity, unit_id)
                        VALUES (%s, %s, %s, %s);
                    """
                    cur.execute(sql_insert_link, (new_recipe_id, ingredient_id, row['quantity'], unit_id))
                
                print(f"Inserted {len(ingredients_df)} ingredients for recipe ID {new_recipe_id}.")
            
            conn.commit()
            print("Transaction successful!")
    
except (Exception, psycopg2.DatabaseError) as error:
    print(f"An error occurred: {error}")
    if conn is not None:
        print("Rolling back transaction.")
        conn.rollback()
    
finally:
    if conn is not None:
        conn.close()
        print("Database connection closed.")

Connecting to the database...
Recipe 'Biscuits and Sausage Gravy' doesn't exist yet. Inserting...
Created new recipe with ID: 4
Inserting ingredients...
'Biscuits' not found in 'ingredients'. Creating new entry...
'Bulk Breakfast Sausage' not found in 'ingredients'. Creating new entry...
'lbs' not found in 'units'. Creating new entry...
'Eggs - fried' not found in 'ingredients'. Creating new entry...
'Milk' not found in 'ingredients'. Creating new entry...
'cup' not found in 'units'. Creating new entry...
'Pepper' not found in 'ingredients'. Creating new entry...
'taste' not found in 'units'. Creating new entry...
'Roux' not found in 'ingredients'. Creating new entry...
'Salt' not found in 'ingredients'. Creating new entry...
Inserted 7 ingredients for recipe ID 4.
Transaction successful!
Database connection closed.
